In [1]:
from dotenv import load_dotenv
from binance.client import Client
import os
import pandas as pd
import numpy as np

load_dotenv()
BINANCE_KEY = os.getenv("BINANCE_KEY")
BINANCE_SECRET = os.getenv("BINANCE_SECRET")
client = Client(
            api_key=BINANCE_KEY,
            api_secret=BINANCE_SECRET,
            requests_params={"timeout": 100},
        )

In [3]:
klines = client.futures_continous_klines(
    pair='ETHUSDT',
    contractType='PERPETUAL',
    interval='1m',
    limit=1000
)



In [178]:
df_klines = pd.DataFrame(
    klines,
    columns=[
        "open_time",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "close_time",
        "quote_asset_volume",
        "number_of_trades",
        "taker_buy_base_asset_volume",
        "taker_buy_quote_asset_volume",
        "ignore",
    ],

)
df_klines['open_time'] = pd.to_datetime(df_klines['open_time'], unit='ms')
df_klines['close_time'] = pd.to_datetime(df_klines['close_time'], unit='ms')
df_klines = df_klines.astype(
    dtype={
        "open": "float",
        "high": "float",
        "low": "float",
        "close": "float",
        "volume": "float",
        "quote_asset_volume": "float",
        "number_of_trades": "int",
        "taker_buy_base_asset_volume": "float",
        "taker_buy_quote_asset_volume": "float",
    }
)
df_klines.dtypes
feature_extraction = df_klines[:700].copy()
final_df = df_klines[700:].copy()

In [179]:
feature_extraction.drop(columns=['ignore'], inplace=True)
final_df.drop(columns=['ignore'], inplace=True)

In [170]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.svm import SVR

# KNN REGRESSION

In [171]:
# feature_extraction['high-low'] = feature_extraction['high'] - feature_extraction['low']
# feature_extraction['open-low'] = feature_extraction['open'] - feature_extraction['low']
# feature_extraction['open-high'] = feature_extraction['open'] - feature_extraction['high']
# feature_extraction['qav/vol'] = feature_extraction['quote_asset_volume'] / feature_extraction['volume']
# feature_extraction['open+high-low'] = feature_extraction['open'] + feature_extraction['high'] - feature_extraction['low']

In [172]:
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('SVR', SVR())
])
grid_params = {
    # 'rfr__n_estimators': [100,200,300],
    # 'rfr__criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
    # 'rfr__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    # 'rfr__leaf_size': [10, 20, 30],
    # 'rfr__p': [1, 2,3],
    # 'ridge__alpha': [0.1, 1, 10],
    # 'ridge__fit_intercept': [True, False],
    'SVR__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'SVR__C': [0.1, 1, 10],
}
grid_search = GridSearchCV(
    knn_pipe,
    param_grid=grid_params,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=1,
    n_jobs=-1
)
X_train, X_test, y_train, y_test = train_test_split(
    feature_extraction.drop(columns=['open_time', 'close_time', 'close']),
    feature_extraction['close'],
    test_size=0.2,
    random_state=42
)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse}")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters: {'SVR__C': 10, 'SVR__kernel': 'linear'}
Best score: -0.5778951764573896
Test MSE: 0.5431172061194276


In [173]:
final_df['svr_feature'] = best_model.predict(
    final_df.drop(columns=['open_time', 'close_time', 'close'])
)
final_df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,svr_feature
700,2025-04-16 17:53:00,1561.00,1563.68,1560.22,1561.36,14754.187,2025-04-16 17:53:59.999,2.304648e+07,9733,9011.269,1.407729e+07,1562.748821
701,2025-04-16 17:54:00,1561.35,1563.00,1556.00,1556.74,10103.077,2025-04-16 17:54:59.999,1.575504e+07,8423,3821.342,5.959264e+06,1558.461218
702,2025-04-16 17:55:00,1556.73,1558.29,1554.42,1555.33,22672.146,2025-04-16 17:55:59.999,3.528813e+07,10763,11878.494,1.848910e+07,1556.423779
703,2025-04-16 17:56:00,1555.34,1556.16,1552.04,1554.02,14903.318,2025-04-16 17:56:59.999,2.316152e+07,10039,5772.219,8.970947e+06,1553.654729
704,2025-04-16 17:57:00,1554.02,1554.77,1540.00,1541.25,75509.766,2025-04-16 17:57:59.999,1.168266e+08,33019,22027.376,3.409116e+07,1543.072928


# Calc'd features

In [174]:
final_df['high-low'] = final_df['high'] - final_df['low']
final_df['open-low'] = final_df['open'] - final_df['low']
final_df['open-high'] = final_df['open'] - final_df['high']
final_df['qav/vol'] = final_df['quote_asset_volume'] / final_df['volume']
final_df['open+high-low'] = final_df['open'] + final_df['high'] - final_df['low']
# final_df['high/low'] = final_df['high'] / final_df['low']
# final_df['open/low'] = final_df['open'] / final_df['low']
# final_df['open/high'] = final_df['open'] / final_df['high']

In [180]:
final_df["sma_20"] = final_df['close'].rolling(window=20).mean()
final_df["sma_50"] = final_df['close'].rolling(window=50).mean()

# CAT FEATURES

In [175]:
final_df['high_gt_1std_open50'] = (final_df['high'] > (final_df['open'] + 0.5*final_df['open'].std())).astype(int)

In [181]:


X_train, X_test, y_train, y_test = train_test_split(
    final_df.drop(columns=['open_time', 'close_time', 'close']),
    final_df['close'],
    test_size=0.2,
    random_state=42
)

pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', XGBRegressor(objective='reg:squarederror'))
])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse}")

feature_importances = pipeline.named_steps['model'].feature_importances_

importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print(importance_df)

RMSE: 1.480913926715554
                        Feature  Importance
1                          high    0.906478
2                           low    0.091405
9                        sma_50    0.000450
8                        sma_20    0.000366
5              number_of_trades    0.000284
6   taker_buy_base_asset_volume    0.000257
0                          open    0.000237
3                        volume    0.000208
7  taker_buy_quote_asset_volume    0.000163
4            quote_asset_volume    0.000154


In [177]:
csv_path = "feature_importance_runs.csv"

# Determine the run number
if os.path.exists(csv_path):
    old = pd.read_csv(csv_path)
    run = old["run"].max() + 1
else:
    run = 0

# Add run column
importance_df["run"] = run

# Append to CSV
importance_df.to_csv(csv_path, mode="a", header=not os.path.exists(csv_path), index=False)

In [183]:
feat_df = pd.read_csv(csv_path)

feat_df.groupby('Feature').mean().sort_values('Importance', ascending=False)

,Importance,run
Feature,,
high,0.652073,7.500000
svr_feature,0.329859,15.000000
ridge_feature,0.315698,13.500000
rfr_feature,0.237851,10.500000
qav/vol,0.203988,9.818182
low,0.060064,7.500000
open+high-low,0.047676,11.142857
knn_feature,0.013293,4.571429
open/low,0.000828,5.000000


additional features: svr, ridge, rfr, qav/vol

would require a pre-training df to not have data leakage

In [202]:
funding_params = {
            "symbol": 'ETHUSDT',
            "limit": 1000
            # "startTime": int(start_time.timestamp() * 1000),
            # "endTime": int(end_time.timestamp() * 1000),
            # "interval": interval,
        }
client.futures_funding_rate(
    funding_params
            # time_col="fundingTime",
            # resample_rule=formatted_resample_rule,
            # cols_to_keep=["fundingRate"],
            # value_cast={"fundingRate": float},
)

TypeError: Client.futures_funding_rate() takes 1 positional argument but 2 were given

In [200]:
client.futures_funding_rate(
symbol='BTCUSDT',
limit=1
)

[{'symbol': 'BTCUSDT',
  'fundingTime': 1744848000001,
  'fundingRate': '0.00001404',
  'markPrice': '83994.18360000'}]

In [207]:
client.futures_open_interest_hist(
    symbol='BTCUSDT', limit=1, period='5m'
)

[{'symbol': 'BTCUSDT',
  'sumOpenInterest': '76322.68800000',
  'sumOpenInterestValue': '6421200325.12909800',
  'timestamp': 1744851300000}]